In [ ]:
#@title 🡐 **First thing first! Please run this hidden cell (click on that `▶` button)!** { form-width: "50%", display-mode: "form" }
%%capture
!git clone -l -s https://github.com/monacotime/DeepForSpeed_decoy.git DeepForSpeed
!pip3 install pytesseract
!sudo apt update
!sudo apt install tesseract-ocr
!sudo apt install libtesseract-dev
import tensorflow as tf
import numpy as np
import pandas as pd
from termcolor import colored
import pytesseract
import cv2
from PIL import Image
from google.colab import files
from io import BytesIO
from matplotlib.pyplot import imshow, show, axis, subplots
%matplotlib inline
plate_size = 400,200
img_height,img_width = 500,500
input_shape = (500, 1000, 1)
input_size = 416
iou = 0.4 #iou threshold
score = 0.4 #score threshold
class_names = ["plate"]
model_path = "/content/DeepForSpeed/yolov3-tiny-deepforspeed.tflite"
def uploadpics():
    image1 = [np.array(Image.open(BytesIO(im1[i])).convert("L").resize((img_width,img_height))) for i in im1]
    image2 = [np.array(Image.open(BytesIO(im2[i])).convert("L").resize((img_width,img_height))) for i in im2]
    comb = np.concatenate((image1[0],image2[0]), axis = 1)
    test_ds = np.expand_dims(comb.reshape(input_shape), 0) / 255.
    return test_ds
def filter_boxes(box_xywh, scores, score_threshold=0.4, input_shape = tf.constant([416,416])):
    scores_max = tf.math.reduce_max(scores, axis=-1)
    mask = scores_max >= score_threshold
    class_boxes = tf.boolean_mask(box_xywh, mask)
    pred_conf = tf.boolean_mask(scores, mask)
    class_boxes = tf.reshape(class_boxes, [tf.shape(scores)[0], -1, tf.shape(class_boxes)[-1]])
    pred_conf = tf.reshape(pred_conf, [tf.shape(scores)[0], -1, tf.shape(pred_conf)[-1]])
    box_xy, box_wh = tf.split(class_boxes, (2, 2), axis=-1)
    input_shape = tf.cast(input_shape, dtype=tf.float32)
    box_yx = box_xy[..., ::-1]
    box_hw = box_wh[..., ::-1]
    box_mins = (box_yx - (box_hw / 2.)) / input_shape
    box_maxes = (box_yx + (box_hw / 2.)) / input_shape
    boxes = tf.concat([box_mins[..., 0:1], box_mins[..., 1:2], box_maxes[..., 0:1], box_maxes[..., 1:2]], axis=-1)
    return (boxes, pred_conf)
def find_plate():
    for i in im1 : orig_img = np.array(Image.open(BytesIO(im1[i])).convert("RGB").getdata(), dtype= "float32").reshape(1000, 1000, 3)
    image_data = cv2.resize(orig_img, (input_size, input_size))
    image_data = image_data / 255.
    image_data = image_data[np.newaxis, ...].astype(np.float32)
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    interpreter.set_tensor(input_details[0]['index'], image_data)
    interpreter.invoke()
    pred = [interpreter.get_tensor(output_details[i]['index']) for i in range(len(output_details))]
    boxes, pred_conf = filter_boxes(pred[1], pred[0], score_threshold=0.4, input_shape=tf.constant([input_size, input_size]))
    b_boxes, *_ = tf.image.combined_non_max_suppression(
        boxes=tf.reshape(boxes, (tf.shape(boxes)[0], -1, 1, 4)),
        scores=tf.reshape(
            pred_conf, (tf.shape(pred_conf)[0], -1, tf.shape(pred_conf)[-1])),
        max_output_size_per_class=50,
        max_total_size=50,
        iou_threshold=iou,
        score_threshold=score)
    crp = np.uint8(b_crop(orig_img, b_boxes.numpy()))
    return orig_img, crp
def b_crop(image, boxes):
    image_h, image_w, _ = map(lambda x: x * 1, image.shape)
    coor = boxes[0][0]
    fac = 20
    return image[int(coor[0] * image_h)+fac:int(coor[2] * image_h)-fac, 
                 int(coor[1] * image_w)+fac:int(coor[3] * image_w)-fac, :]
def make_receipt():
    print(f"Hello {person.Name},",
        colored("\nYou done messed up now! 😈", 'red'),
        "\nYou were caught running at speeds of:", '\033[1m' + '\033[94m' + str(int(speed)), "km/hr")
    print(colored('\nYour details and the proof below: 👁️‍🗨️👄👁️‍🗨️', 'red'))
    print(colored(person, "green"))
    imshow(test_ds[0].squeeze(), cmap = "gray"); axis("off"); show()
model = tf.keras.models.load_model("/content/DeepForSpeed/DeepForSpeed_model_no_dropout.h5")
df = pd.read_csv("/content/DeepForSpeed/License.csv", index_col = "License")

#### **Now while that's running, I'll explain a little bit of what you must do to run this demo.** <br>

1. Each block of text you see is called a `cell`. You can run them by pressing the " `▶` (run cell)" button to the left of it.

2. Every cell will have a small description of what the code in that cell does. <br> And you can: <br>
`double click that description` to ***unhide*** the code 🙉<br>
`double click that description` again to ***hide*** the code 🙈

So just keep going down the page as you run every cell `from top to bottom` 

### And dont forget to have fun! 🥳


In [ ]:
#@title 🡐 **Here you upload two images**:  <br> First image: `Before` picture of the car as it passes by 🚕<br> Second image: `After` picture of the car taken **0.25** ⏲️ seconds after the first picture 🛫{ display-mode: "form" }
try:
    im1 = files.upload()
    im2 = files.upload()
    test_ds = uploadpics()
except:
    print(colored("Sorry, you have run to an error during upload! 😢📝 \nPlease re-run this cell and try again!","red", attrs=['reverse', 'blink', 'bold']))

In [ ]:
#@title 🡐 **This cell simply:** <br>Shows you the two images you uploaded 🖼️<br> And **`PREDICTS`** how fast the car is moving in those pictures! 🏎️
speed = model.predict(test_ds)[0][0]
imshow(test_ds[0].squeeze(), cmap = "gray"); axis("off"); show()
print('\033[1m' + '\033[94m' + ">THE SPEED OF THE CAR IS:", speed,"KM/HR")

In [ ]:
#@title 🡐 **Using EPIC awesomeness ✨:** <br> It `finds` the number plate on your car 💳 <br> And `extracts` the text from the image 📜
plate = find_plate()[1]
imshow(plate); axis("off"); show()
text = pytesseract.image_to_string(Image.fromarray(plate).resize((600,300), Image.ANTIALIAS))
print('\033[1m' + '\033[94m' + ">License number is: ", text)#.split("\n")[0]

In [ ]:
#@title 🡐 **ITS TIME TO CATCH THE BAD GUY! 🕵️**
try:
    person = df.loc[text.split("\n")[0]]
    # person = df.loc["myname"]
    make_receipt()
except:
    print(colored("Sorry, This person does not seem to be in our database records! 😢📝","red", attrs=['reverse', 'blink', 'bold']))